In [1]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs
#from google.colab import files
from tqdm import *
#courses = pd.read_csv('courses_all.csv')
#career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])

In [64]:
eng_list = []

In [10]:
for i in tqdm(range(82)):
    page = rq.get('https://medicine.careers360.com/colleges/list-of-medicine-colleges-in-India?page='+str(i))
    soup = bs(page.content,'html.parser')
    vals = soup.find_all('div',class_='title')
    for val in vals:
        try:
            eng_list.append(val.a['href'])
        except:
            pass

100%|██████████| 82/82 [01:08<00:00,  1.20it/s]


In [2]:
courses = pd.read_csv('medical_courses.csv')

In [4]:
courses = pd.DataFrame(columns=[0,1,2])

In [5]:
idx = 0
for link in tqdm(courses1['0']):
    try:
        page = rq.get(link)
        soup = bs(page.content,'html.parser')
        courses.loc[idx]=[link,soup.find_all('h4',class_='blockSubHeading')[0].get_text()[-2:],'https://medicine.careers360.com'+soup.find_all('ul',class_='list-tabs')[0].find_all('li')[1].a['href']]
        idx+=1
    except:
        pass

100%|██████████| 2454/2454 [21:10<00:00,  1.93it/s]


In [3]:
courses.head()

,Unnamed: 0,0,1,2
0,0,https://medicine.careers360.com/colleges/all-i...,19,https://medicine.careers360.com/colleges/all-i...
1,1,https://medicine.careers360.com/Medicine/Maula...,9,https://medicine.careers360.com/colleges/maula...
2,2,https://medicine.careers360.com/colleges/guru-...,11,https://medicine.careers360.com/guru-gobind-si...
3,3,https://medicine.careers360.com/colleges/armed...,15,https://medicine.careers360.com/colleges/armed...
4,4,https://medicine.careers360.com/colleges/jamia...,4,https://medicine.careers360.com/colleges/jamia...


In [3]:
from threading import Thread
def scrapper(start, end):
    career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])
    indx = 0
    cnt = start - 1
    for header_course in tqdm(courses['2'][start:end]):
        cnt += 1
        try:
            num = int(np.ceil(int(courses['1'][cnt])/10))
        except:
            continue
    #     print(f'num = {num}')
        for page_num in tqdm(range(num)):
            link = header_course + '?levelId=all&courseNid=all-all&page='+str(page_num)
    #         print(link)
            page = rq.get(link)
            course_soup = bs(page.content, 'html.parser')
        for course_link in course_soup.find_all('span',class_='readmore'):
            course_link = 'https://medicine.careers360.com' + course_link.a['href']
            #             print(f'course_link = {course_link}')
            page1 = rq.get(course_link)
            soup = bs(page1.content,'html.parser')
            career360_courses.loc[indx]=None

            try:
                career360_courses.loc[indx]['Course'] = soup.find_all('h2',class_='block-title')[0].get_text()
                career360_courses.loc[indx]['College']=soup.find_all('h1',class_='titleNameCol')[0].get_text()
                career360_courses.loc[indx]['City'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[0].get_text()
                career360_courses.loc[indx]['State'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[1].get_text()
            except:
                pass

            try:
                career360_courses.loc[indx]['Eligiblity'] = soup.find_all('span',class_='more-eligibility')[0].get_text()[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Eligiblity'] = soup.find_all('div',class_='default-elig')[0].get_text()
                except:
                    career360_courses.loc[indx]['Eligiblity'] = None

            try:
                career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('span',class_='moreCourseDetails')[0].p.get_text())[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('div',class_='default-course-details')[0].get_text())
                except:
                    career360_courses.loc[indx]['Course_Detail'] = None

            for row in soup.find_all('div',class_='coursesPageLableInnerSec'):
                career360_courses.loc[indx][row.strong.get_text()] = row.p.get_text()
            indx+=1
    career360_courses.to_csv(f'career360_scrap_dm_medical({start}-{end}).csv')

In [4]:
siz = courses.shape[0]
numer = int(siz/20)
prev = 0
def threader():
    threads = []
    for i in range(20):
        if i!=19:
            thread = Thread(target=scrapper, args = (i*numer,(i+1)*numer))
            threads.append(thread)
            thread.start()
        else:
            thread = Thread(target=scrapper, args = (19*numer,siz))
            threads.append(thread)
            thread.start()        
    
    for thread in threads:  # iterates over the threads
        thread.join()
                    
    print('done')

In [9]:
threader()

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]



  0%|          | 0/122 [00:00<?, ?it/s]




  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/122 [00:00<?, ?it/s]






  0%|          | 0/1 [00:00<?, ?it/s]







  0%|          | 0/122 [00:00<?, ?it/s]








  0%|          | 0/1 [00:00<?, ?it/s]









  0%|          | 0/122 [00:00<?, ?it/s]










  0%|          | 0/1 [00:00<?, ?it/s]











  0%|          | 0/122 [00:00<?, ?it/s]












  0%|          | 0/1 [00:00<?, ?it/s]













  0%|          | 0/122 [00:00<?, ?it/s]














  0%|          | 0/1 [00:00<?, ?it/s]















  0%|          | 0/122 [00:00<?, ?it/s]
















  0%|          | 0/1 [00:00<?, ?it/s]

















  0%|          | 0/122 [00:00<?, ?it/s]


















  0%|          | 0/1 [00:00<?, ?it/s]



















  0%|          | 0/122 [00:00<?, ?it/s]




















  0%|     

  0%|          | 0/1 [00:00<?, ?it/s]


  1%|          | 1/122 [00:06<12:15,  6.08s/it]




























  0%|          | 0/1 [00:00<?, ?it/s]













 67%|██████▋   | 2/3 [00:06<00:03,  3.11s/it]



















  2%|▏         | 2/122 [00:06<06:20,  3.17s/it]





























  0%|          | 0/1 [00:00<?, ?it/s]

























100%|██████████| 1/1 [00:02<00:00,  2.60s/it]































  1%|          | 1/122 [00:06<13:37,  6.76s/it]





























  0%|          | 0/1 [00:00<?, ?it/s]




















  1%|          | 1/122 [00:06<13:07,  6.51s/it]






























  0%|          | 0/1 [00:00<?, ?it/s]
























100%|██████████| 1/1 [00:03<00:00,  3.04s/it]
















































100%|██████████| 2/2 [00:06<00:00,  3.25s/it]
































  1%|          | 1/122 [00:06<13:41,  6.79s/it]





























  0%|          | 0/1 [00:00

 50%|█████     | 1/2 [00:03<00:03,  3.12s/it]














 71%|███████▏  | 5/7 [00:13<00:05,  2.68s/it]







  2%|▏         | 2/122 [00:13<13:32,  6.77s/it]

























  0%|          | 0/2 [00:00<?, ?it/s]





  2%|▏         | 2/122 [00:13<13:36,  6.81s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]























100%|██████████| 1/1 [00:02<00:00,  2.77s/it]

















100%|██████████| 4/4 [00:13<00:00,  3.39s/it]Exception in thread Thread-18:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-7-d5fdf0199d8c>", line 13, in scrapper
    for page_num in tqdm(range(num)):
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 990, in __iter__
    self.close()
  File "/home/ubuntu

  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 3/3 [00:06<00:00,  2.07s/it]























  3%|▎         | 4/122 [00:26<13:00,  6.62s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]











  3%|▎         | 4/122 [00:26<12:56,  6.58s/it]



























  0%|          | 0/1 [00:00<?, ?it/s]







  2%|▏         | 3/122 [00:26<17:32,  8.84s/it]




























  0%|          | 0/2 [00:00<?, ?it/s]




















100%|██████████| 1/1 [00:01<00:00,  1.40s/it]








































100%|██████████| 1/1 [00:02<00:00,  2.40s/it]










































100%|██████████| 1/1 [00:01<00:00,  1.78s/it]
































  2%|▏         | 3/122 [00:28<18:57,  9.56s/it]









  3%|▎         | 4/122 [00:28<14:13,  7.24s/it]


























  0%|          | 0/2 [00:00<?, ?it/s]






















100%|██████████| 1/1 [00:02<00:00,  2.97s/it]










100%|██████████| 1/1 [00:02<00:00,  2.91s/it]














































100%|██████████| 1/1 [00:03<00:00,  3.19s/it]




























  4%|▍         | 5/122 [00:38<14:56,  7.66s/it]

























  0%|          | 0/1 [00:00<?, ?it/s]






  4%|▍         | 5/122 [00:38<14:56,  7.66s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 3/3 [00:08<00:00,  2.88s/it]











































100%|██████████| 1/1 [00:02<00:00,  2.99s/it]













































100%|██████████| 1/1 [00:03<00:00,  3.18s/it]

































  5%|▍         | 6/122 [00:38<12:20,  6.38s/it]
























  0%|          | 0/2 [00:00<?, ?it/s]
















  4%|▍         | 5/122 [00:38<14:53,  7.64s/it]

























  0%|          | 0/1 [00:00<?, ?it/s]




  7%|▋         | 9/122 [00:38<08:03,  4.28s/it]


























  0%|       

...

100%|██████████| 1/1 [00:01<00:00,  1.87s/it]






























100%|██████████| 4/4 [00:06<00:00,  1.58s/it]





























100%|██████████| 1/1 [00:02<00:00,  2.73s/it]






















 80%|████████  | 98/122 [13:39<03:20,  8.36s/it]














  0%|          | 0/2 [00:00<?, ?it/s]




 75%|███████▌  | 92/122 [13:40<04:27,  8.92s/it]















 52%|█████▏    | 64/122 [13:41<12:24, 12.83s/it]
















  0%|          | 0/1 [00:00<?, ?it/s]














 50%|█████     | 1/2 [00:01<00:01,  1.72s/it]















100%|██████████| 1/1 [00:02<00:00,  2.33s/it]































100%|██████████| 1/1 [00:02<00:00,  2.08s/it]






























100%|██████████| 2/2 [00:04<00:00,  2.05s/it]






















 81%|████████  | 99/122 [13:43<03:11,  8.32s/it]














  0%|          | 0/1 [00:00<?, ?it/s]














100%|██████████| 1/1 [00:01<00:00,  1.50s/it]















 53%|█████▎    | 65/122 [13:47<12:05,

100%|██████████| 1/1 [00:01<00:00,  1.77s/it]






























100%|██████████| 7/7 [00:11<00:00,  1.60s/it]



















 84%|████████▎ | 102/122 [15:03<02:57,  8.86s/it]















  0%|          | 0/1 [00:00<?, ?it/s]







 95%|█████████▌| 116/122 [15:03<00:46,  7.79s/it]
















  0%|          | 0/1 [00:00<?, ?it/s]














 40%|████      | 2/5 [00:03<00:04,  1.50s/it]
















100%|██████████| 1/1 [00:01<00:00,  1.68s/it]
































100%|██████████| 1/1 [00:01<00:00,  1.83s/it]
















 59%|█████▉    | 72/122 [15:05<10:28, 12.58s/it]















  0%|          | 0/1 [00:00<?, ?it/s]



 71%|███████▏  | 87/122 [15:05<06:04, 10.41s/it]
















  0%|          | 0/1 [00:00<?, ?it/s]














 60%|██████    | 3/5 [00:04<00:02,  1.49s/it]




 84%|████████▍ | 103/122 [15:06<02:47,  8.81s/it]

















  0%|          | 0/1 [00:00<?, ?it/s]















100%|██████████| 1/1 [00:01<00:00,  1


 65%|██████▍   | 79/122 [16:19<08:52, 12.39s/it]













  0%|          | 0/1 [00:00<?, ?it/s]



 82%|████████▏ | 100/122 [16:19<03:35,  9.80s/it]














  0%|          | 0/1 [00:00<?, ?it/s]













100%|██████████| 1/1 [00:01<00:00,  1.14s/it]



























100%|██████████| 1/1 [00:01<00:00,  1.58s/it]

















 83%|████████▎ | 101/122 [16:23<03:24,  9.74s/it]













  0%|          | 0/3 [00:00<?, ?it/s]













 66%|██████▌   | 80/122 [16:26<08:37, 12.33s/it]














  0%|          | 0/2 [00:00<?, ?it/s]













 67%|██████▋   | 2/3 [00:02<00:01,  1.42s/it]














 50%|█████     | 1/2 [00:01<00:01,  1.22s/it]













100%|██████████| 3/3 [00:04<00:00,  1.57s/it]

















 84%|████████▎ | 102/122 [16:28<03:13,  9.69s/it]














  0%|          | 0/1 [00:00<?, ?it/s]




 90%|█████████ | 110/122 [16:28<01:47,  8.98s/it]















  0%|          | 0/1 [00:00<?, ?it/s]













100%|████████


 80%|████████  | 98/122 [19:13<04:42, 11.77s/it]











  0%|          | 0/1 [00:00<?, ?it/s]











100%|██████████| 1/1 [00:01<00:00,  1.29s/it]












 81%|████████  | 99/122 [19:15<04:28, 11.68s/it]











  0%|          | 0/1 [00:00<?, ?it/s]











100%|██████████| 1/1 [00:02<00:00,  2.26s/it]












 82%|████████▏ | 100/122 [19:28<04:17, 11.69s/it]











  0%|          | 0/1 [00:00<?, ?it/s]











100%|██████████| 1/1 [00:02<00:00,  2.08s/it]












 83%|████████▎ | 101/122 [19:33<04:04, 11.62s/it]











  0%|          | 0/2 [00:00<?, ?it/s]











 50%|█████     | 1/2 [00:02<00:02,  2.10s/it]











100%|██████████| 2/2 [00:04<00:00,  2.30s/it]












 84%|████████▎ | 102/122 [19:48<03:53, 11.65s/it]











  0%|          | 0/1 [00:00<?, ?it/s]











100%|██████████| 1/1 [00:01<00:00,  1.64s/it]












 84%|████████▍ | 103/122 [20:00<03:41, 11.65s/it]











  0%|          | 0/1 [00:00<?, ?it/s]







done


In [6]:
courses.to_csv('medical_courses.csv')

In [6]:
scrapper(732,976)

 89%|████████▉ | 8/9 [00:03<00:00,  2.56it/s]


100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


100%|██████████| 1/1 [00:02<00:00,  2.04s/it]


100%|██████████| 3/3 [00:04<00:00,  1.58s/it]


100%|██████████| 244/244 [15:13<00:00,  3.74s/it]


In [7]:
scrapper(1098,1220)

 50%|█████     | 4/8 [00:05<00:05,  1.43s/it]


 91%|█████████ | 111/122 [10:42<01:03,  5.79s/it]


100%|██████████| 122/122 [11:49<00:00,  5.81s/it]


In [8]:
scrapper(1464,1830)

100%|██████████| 10/10 [00:16<00:00,  1.65s/it]


100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


  0%|          | 0/1 [00:00<?, ?it/s]


 82%|████████▏ | 301/366 [16:28<03:33,  3.28s/it]


 50%|█████     | 2/4 [00:02<00:02,  1.33s/it]


100%|██████████| 366/366 [19:03<00:00,  3.12s/it]


In [9]:
scrapper(2074,2318)

 75%|███████▌  | 3/4 [00:01<00:00,  2.08it/s]


 60%|██████    | 147/244 [03:03<02:01,  1.25s/it]


100%|██████████| 1/1 [00:01<00:00,  1.45s/it]


100%|██████████| 244/244 [07:04<00:00,  1.74s/it]


In [13]:
indx

NameError: name 'indx' is not defined